In [1]:
#Importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. Configurações Iniciais

In [2]:
arquivos = {
    'idle_differential_bruto.csv': 0,
    'gaming_differential_bruto.csv': 1,
    'obstrucao_differential_bruto.csv': 2
}
colunas = ['timestamp', 't_ex', 'u_ex', 't_amb', 'u_amb', 'delta_t', 'delta_u', 'label_str']

# 2. Carregamento e Rotulagem

In [3]:
dfs = []
for nome_arq, classe in arquivos.items():
    try:
        # Carrega forçando os nomes das colunas (já que os arquivos brutos não têm header)
        df_temp = pd.read_csv(nome_arq, names=colunas, header=None)
        df_temp['classe'] = classe
        dfs.append(df_temp)
        print(f"Lido: {nome_arq} | Amostras: {len(df_temp)}")
    except FileNotFoundError:
        print(f"ERRO: Arquivo {nome_arq} não encontrado!")
# Concatenar todos
df_completo = pd.concat(dfs, ignore_index=True)

Lido: idle_differential_bruto.csv | Amostras: 1655
Lido: gaming_differential_bruto.csv | Amostras: 2549
Lido: obstrucao_differential_bruto.csv | Amostras: 1719


# 3. Balanceamento (Down-sampling)

In [4]:
menor_classe = df_completo['classe'].value_counts().min()
df_balanceado = df_completo.groupby('classe').sample(n=menor_classe, random_state=42)

print(f"\nDataset balanceado para {menor_classe} amostras por classe.")


Dataset balanceado para 1655 amostras por classe.


# 4. Extração de Características e Normalização

In [5]:
X = df_balanceado[['delta_t', 'delta_u']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. GERADOR DE CONSTANTES PARA O C (BITDOGLAB)

In [6]:
mean = scaler.mean_
std = np.sqrt(scaler.var_)

print("\n" + "="*45)
print("   COPIE E COLE NO SEU CÓDIGO C (CONSTANTES DELTA)   ")
print("="*45)
print(f"const float MEAN_DELTA_T = {mean[0]:.8f}f;")
print(f"const float STD_DELTA_T  = {std[0]:.8f}f;")
print(f"const float MEAN_DELTA_U = {mean[1]:.8f}f;")
print(f"const float STD_DELTA_U  = {std[1]:.8f}f;")
print("="*45)


   COPIE E COLE NO SEU CÓDIGO C (CONSTANTES DELTA)   
const float MEAN_DELTA_T = 15.65084995f;
const float STD_DELTA_T  = 6.99135624f;
const float MEAN_DELTA_U = -38.53349043f;
const float STD_DELTA_U  = 12.27171964f;


# 6. Salvar para o treino

In [7]:
df_balanceado.to_csv('dataset_pronto_treino.csv', index=False)
print("\nArquivo 'dataset_pronto_treino.csv' gerado com sucesso!")


Arquivo 'dataset_pronto_treino.csv' gerado com sucesso!
